### Person Bot Version 0.1

Hey! Created this template for create a basic bot which you can customize with a system prompt, and can generate images. Feel free to use this to create a chatbot clone of yourself or your friends(with their permission ofc) :)

#### Importing Libraries

In [14]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

Using ```dotenv ``` and  ```load_dotenv``` to use and load enviromental variables but mainly to load up the API keys

In [15]:
#Loading ENV Variables

load_dotenv(override = True)
openai_api_key = os.getenv("OPENAI_API_KEY")


```load_dotenv``` loads up the .env file.

```os.getenv("...")``` retrieves the '...' value to a variable openai_api_key


#### Setting Up Model

In [31]:
MODEL = 'gpt-4o'

Sets openai object and specifies what model to use

#### Creating a System Prompt

System prompt is needed to define how the AI Model behaves

In [ ]:
system_prompt = "[PUT YOUR CHARACTER DESCRIPTION HERE]"

#### Defining Generate Image Function

In [20]:
import base64
from io import BytesIO
from PIL import Image
import json

```import base64```
Used to encode and decode date in Base64 format

```from io import BytesIO```

```io``` allows you to handle streams of data

```BytesIO``` Creates an in-memory binary stream, treating bytes as a file-like object. Allowing to handle the image data without writing to disk

In [30]:
def rocks():
    image_response = openai.images.generate(
        model = "dall-e-3",
        prompt = "An image of a nice pebble or rock, with interesting shapes, held by a firm masculine hand, HD, iphone camera",
        size = "1024x1024",
        n = 1,
        response_format= "b64_json")
    
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

##### Retrieving image data
``` image_base64 = image_response.data[0].bs64_json```

Get's the image data from the image_response object

```image_response.data[0]```

accesses the image data index from the image_response object

```.bs64_json``` 

contains the image data in JSON format

##### Converting image data to binary

```base64.b64decode(image_base64)```

Decodes string into binary image data

##### Loading image

```return Image.open(BytesIO(image_data))```

```BytesIO(image_data)``` 

Creates an in-memory binary stream (file - like object) from the binary image data (image)data

```Image.open()``` 

From PIL library, which opens an image from a file-like object




#### Tool Dictionary

Creates a dictionary, containing information about a specified tool. Rocks is a place holder, you can rename it based on your function

In [22]:

tools = [
    {
        "type" : "function",
        "function": {
            "name": "rocks",
            "description": "Generates an image of a nice image or rock held by you.",
            "parameters": {} 
        }
    }
]
    

#### Handling Tool Calls

In [23]:
def handle_tool_call(message):
    if not message.tool_calls:
        return {"role": "tool", "tool_call_id": None, "content": "No tool call found."}

    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name

    response = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": ""
    }

    if function_name == "rocks":
        image = rocks()  
        response["content"] = "Look at this cool rock i found"
        return response, image
 
    response["content"] = f"Unknown function: {function_name}"
    return response, None 

```if not message.tool_calls```

Checks to see if message.tool_calls exists or is empty. If not:

```return {"role": "tool", "tool_call_id": None, "content": "No tool call found."}```

Which creates a dictionary, showing that it's coming from a tool and that there is no ID as no tool exits, and returns a message showing that nothing was called

```tool_call = message.tool_calls[0]```

Retrieves first tool call from message.tool_calls, get's the function name and stores it in ```function_name```

```response = {...}```

Creates a response dictionary, like before, content is left empty to be changed later

```if function_name == "rocks":```
performs what the ufnction is supposed to do

```response["content"] = f"Unknown function: {function_name}"```

Returns the response dictionary with the content text

### Chat Function

In [26]:
import openai

In [ ]:
client = openai.OpenAI()

def chat(history):
    messages = [{"role":"system", "content":system_prompt}] + history
    response = client.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools
    )

    image = None

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_response, image = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)

        response = client.chat.completions.create(
            model = "gpt-4",
            messages = messages
        )
    
    reply = response.choices[0].message.content
    history.append({"role": "assistant", "content": reply})

    return history, image


```response.choices[0].finish_reason = "tool_calls```

Checks if response has a request to call a tool

```tool_response, image = handle_tool_call(message)```

Returns response and image from handle_tool_call function



#### Gradio UI

UI made with gradio

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Talk To Ahmad:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7907

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Nick-Uni\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\Nick-Uni\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\Nick-Uni\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 2051, in process_api
    result = await self.call_function(
  File "c:\Users\Nick-Uni\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1598, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\Nick-Uni\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\Nick-Uni\AppData\Local\